# 🍫Tune your RAG data pipeline and evaluate its performance

> ⚠️ This notebook can be run on your local machine or on a virtual machine and requires [Docker Compose](https://docs.docker.com/desktop/).
> Please note that it is not compatible with Google Colab as the latter does not support Docker.

In this notebook we demonstrate how to iteratively evaluate and tune a Retrieval-Augmented Generation (RAG) system using [Fondant](https://fondant.ai).

We will:

1. Set up a [Weaviate](https://weaviate.io/platform) vector store
2. Define a parameter set to test
3. Run a Fondant pipeline with those parameters to index our documents into the vector store
4. Run a Fondant pipeline with those parameters to evaluate the performance
5. Inspect the evaluation results and data between each processing step
6. Repeat step 2 - 5 until we're happy with the results

<div align="center">
<img src="../art/iteration.png" width="1000"/>
</div>

## Set up environment

> ⚠️ This section checks the prerequisites of your environment. Read any errors or warnings carefully.

Ensure a **Python between version 3.8 and 3.10** is available

In [1]:
import sys
if sys.version_info < (3, 8, 0) or sys.version_info >= (3, 11, 0):
    raise Exception(f"A Python version between 3.8 and 3.10 is required. You are running {sys.version}")

Check if **docker compose** is installed and the **docker daemon** is running

In [2]:
!docker compose version

Docker Compose version v2.19.1


Install Fondant framework

In [4]:
!pip install -q -r ../requirements.txt --disable-pip-version-check && echo "Success"

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datasets 2.16.1 requires fsspec[http]<=2023.10.0,>=2023.1.0, but you have fsspec 2023.12.2 which is incompatible.
Success


**Check if GPU is available**

In [3]:
import logging
import subprocess

try:
    subprocess.check_output('nvidia-smi')
    logging.info("Found GPU, using it!")
    number_of_accelerators = 1
    accelerator_name = "GPU"
except Exception:
    logging.warning("We recommend to run this pipeline on a GPU, but none could be found, using CPU instead")
    number_of_accelerators = None
    accelerator_name = None

## Spin up the Weaviate vector store

> ⚠️ For **Apple M1/M2** chip users:
> 
> - In Docker Desktop Dashboard `Settings -> Features in development`, make sure to **un**check `Use containerd` for pulling and storing images. More info [here](https://docs.docker.com/desktop/settings/mac/#beta-features)
> - Make sure that Docker uses linux/amd64 platform and not arm64 (cell below should take care of that)

Run **Weaviate** with Docker compose

In [4]:
!docker compose -f weaviate_service/docker-compose.yaml up --detach

[+] Running 2/0
 ✔ Container weaviate_service-weaviate-1       Running                     0.0s 
 ✔ Container weaviate_service-contextionary-1  Running                     0.0s 


Make sure you have **Weaviate client v3**

Make sure the vectorDB is running and accessible

In [5]:
import logging
import weaviate

try:
    local_weaviate_client = weaviate.Client("http://localhost:8081")
    logging.info("Connected to Weaviate instance")
except weaviate.WeaviateStartUpError:
    logging.error("Cannot connect to weaviate instance, is it running?")

/Users/matthias/projects/fondant/fondant-usecase-RAG/.venv/lib/python3.10/site-packages/weaviate/warnings.py:121: DeprecationWarning: Dep005: You are using weaviate-client version 3.25.3. The latest version is 4.4.2.
            Please consider upgrading to the latest version. See https://weaviate.io/developers/weaviate/client-libraries/python for details.
  warnings.warn(


#### Indexing pipeline

`pipeline_index.py` processes text data and loads it into the vector database

<div align="center">
<img src="../art/indexing_ltr.png" width="800"/>
</div>

- [**Load data**](https://github.com/ml6team/fondant/tree/main/components/load_from_parquet): loads data from the Hugging Face Hub
- [**Chunk data**](https://github.com/ml6team/fondant/tree/main/components/chunk_text): divides the text into sections of a certain size and with a certain overlap
- [**Embed chunks**](https://github.com/ml6team/fondant/tree/main/components/embed_text): embeds each chunk as a vector, e.g. using [Cohere](https://cohere.com/embeddings)
- [**Index vector store**](https://github.com/ml6team/fondant/tree/main/components/index_weaviate): writes data and embeddings to the vector store

> 💡 This notebook defaults to the first 1000 rows of the [wikitext](https://huggingface.co/datasets/wikitext) dataset for demonstration purposes, but you can load your own dataset using one the other load components available on the [**Fondant Hub**](https://fondant.ai/en/latest/components/hub/#component-hub) or by creating your own [**custom load component**](https://fondant.ai/en/latest/guides/implement_custom_components/). Keep in mind that changing the dataset implies that you also need to change the evaluation dataset used in the evaluation pipeline. 

#### Create the indexing pipeline

We are reusing the index pipeline from the [indexing notebook](./indexing.ipynb). Therefore, we have extracted the code into a separate file and created a function that parameterizes the entire pipeline. 

In [6]:
import pipeline_index
import utils

# Path where data and artifacts will be stored
BASE_PATH = "./data"
utils.create_directory_if_not_exists(BASE_PATH)

# Parameters shared between indexing and evaluation pipeline
shared_args = {
    "base_path": BASE_PATH,
    "embed_model_provider": "huggingface",
    "embed_model": "all-MiniLM-L6-v2",
    "weaviate_url": f"http://{utils.get_host_ip()}:8081",
    "weaviate_class": "Pipeline1", # Capitalized, avoid special characters (_, =, -, etc.)
}

# Parameters for the indexing pipeline
indexing_args = {
    "n_rows_to_load": 1000,
    "chunk_args": {"chunk_size": 512, "chunk_overlap": 32}
}

# Parameters for the GPU resources
resources_args = {
    "number_of_accelerators": number_of_accelerators,
    "accelerator_name": accelerator_name,
}

indexing_pipeline = pipeline_index.create_pipeline(**shared_args, **indexing_args, **resources_args)

#### Run the indexing pipeline

> 💡 The first time you run a pipeline, you need to **download a docker image for each component** which may take a minute.

> 💡 Use a **GPU** or an external API to speed up the embedding step

> 💡 Steps that have been processed before are **cached** and will be skipped in subsequent runs which speeds up processing.

In [7]:
from fondant.pipeline.runner import DockerRunner

runner = DockerRunner()
runner.run(indexing_pipeline)

INFO:root:Found reference to un-compiled pipeline... compiling
INFO:fondant.pipeline.compiler:Compiling indexing-pipeline to .fondant/compose.yaml
INFO:fondant.pipeline.compiler:Base path found on local system, setting up ./data as mount volume
INFO:fondant.pipeline.pipeline:Sorting pipeline component graph topologically.
INFO:fondant.pipeline.pipeline:All pipeline component specifications match.
INFO:fondant.pipeline.compiler:Compiling service for load_from_hugging_face_hub
INFO:fondant.pipeline.compiler:Compiling service for chunktextcomponent
INFO:fondant.pipeline.compiler:Compiling service for embed_text
INFO:fondant.pipeline.compiler:Compiling service for index_weaviate
INFO:fondant.pipeline.compiler:Successfully compiled to .fondant/compose.yaml
 load_from_hugging_face_hub Pulling 
 embed_text Pulling 
 index_weaviate Pulling 
 chunktextcomponent Pulling 


Starting pipeline run...


 load_from_hugging_face_hub Pulled 
 chunktextcomponent Pulled 
 embed_text Pulled 
 index_weaviate Pulled 
 Container indexing-pipeline-load_from_hugging_face_hub-1  Recreate
 Container indexing-pipeline-load_from_hugging_face_hub-1  Recreated
 Container indexing-pipeline-chunktextcomponent-1  Recreate
 Container indexing-pipeline-chunktextcomponent-1  Recreated
 Container indexing-pipeline-embed_text-1  Recreate
 Container indexing-pipeline-embed_text-1  Recreated
 Container indexing-pipeline-index_weaviate-1  Recreate
 Container indexing-pipeline-index_weaviate-1  Recreated


Attaching to indexing-pipeline-chunktextcomponent-1, indexing-pipeline-embed_text-1, indexing-pipeline-index_weaviate-1, indexing-pipeline-load_from_hugging_face_hub-1


indexing-pipeline-load_from_hugging_face_hub-1  | [2024-02-06 12:18:43,582 | fondant.cli | INFO] Component `LoadFromHubComponent` found in module main
indexing-pipeline-load_from_hugging_face_hub-1  | [2024-02-06 12:18:43,588 | fondant.component.executor | INFO] Dask default local mode will be used for further executions.Our current supported options are limited to 'local' and 'default'.
indexing-pipeline-load_from_hugging_face_hub-1  | [2024-02-06 12:18:43,592 | fondant.component.executor | INFO] Skipping component execution
indexing-pipeline-load_from_hugging_face_hub-1  | [2024-02-06 12:18:43,594 | fondant.component.executor | INFO] Matching execution detected for component. The last execution of the component originated from `indexing-pipeline-20240206095839`.
indexing-pipeline-load_from_hugging_face_hub-1  | [2024-02-06 12:18:43,599 | fondant.component.executor | INFO] Saving output manifest to /data/indexing-pipeline/indexing-pipeline-20240206131838/load_from_hugging_face_hub/man

...

[########################                ] | 60% Completed | 23.57 s
[########################                ] | 60% Completed | 23.67 s


indexing-pipeline-embed_text-1                  | 
indexing-pipeline-embed_text-1                  | 
indexing-pipeline-embed_text-1                  | 
Batches: 100%|██████████| 10/10 [00:23<00:00,  2.36s/it]
indexing-pipeline-embed_text-1                  | [2024-02-06 12:19:44,047 | fondant.component.executor | INFO] Saving output manifest to /data/indexing-pipeline/indexing-pipeline-20240206131838/embed_text/manifest.json
indexing-pipeline-embed_text-1                  | [2024-02-06 12:19:44,048 | fondant.component.executor | INFO] Writing cache key with manifest reference to /data/indexing-pipeline/cache/dd31849c92bae5c6f64b2cca40426e9f.txt


[########################                ] | 60% Completed | 23.78 s
[########################################] | 100% Completed | 23.88 s
indexing-pipeline-embed_text-1 exited with code 0


indexing-pipeline-index_weaviate-1              | [2024-02-06 12:19:48,587 | fondant.cli | INFO] Component `IndexWeaviateComponent` found in module main
indexing-pipeline-index_weaviate-1              | [2024-02-06 12:19:48,598 | fondant.component.executor | INFO] Dask default local mode will be used for further executions.Our current supported options are limited to 'local' and 'default'.
indexing-pipeline-index_weaviate-1              | [2024-02-06 12:19:48,623 | fondant.component.executor | INFO] Previous component `embed_text` is not cached. Invalidating cache for current and subsequent components
indexing-pipeline-index_weaviate-1              | [2024-02-06 12:19:48,623 | fondant.component.executor | INFO] Caching disabled for the component
indexing-pipeline-index_weaviate-1              | [2024-02-06 12:19:48,624 | root | INFO] Executing component
indexing-pipeline-index_weaviate-1              | /usr/local/lib/python3.8/site-packages/weaviate/warnings.py:121: DeprecationWarning:

indexing-pipeline-index_weaviate-1 exited with code 0
Finished pipeline run.


## Evaluation Pipeline

`pipeline_eval.py` evaluates retrieval performance using the questions provided in your test dataset

<div align=center>
<img src="../art/evaluation_ltr.png" width="800"/>
</div>

- [**Load eval data**](https://github.com/ml6team/fondant/tree/main/components/load_from_csv): loads the evaluation dataset (questions) from a csv file
- [**Embed questons**](https://github.com/ml6team/fondant/tree/main/components/embed_text): embeds each question as a vector, e.g. using [Cohere](https://cohere.com/embeddings)
- [**Query vector store**](https://github.com/ml6team/fondant/tree/main/components/retrieve_from_weaviate): retrieves the most relevant chunks for each question from the vector store
- [**Evaluate**](https://github.com/ml6team/fondant/tree/0.8.0/components/evaluate_ragas): evaluates the retrieved chunks for each question, e.g. using [RAGAS](https://docs.ragas.io/en/latest/index.html)
- [**Aggregate**](https://github.com/ml6team/fondant-usecase-RAG/tree/main/src/components/aggregate_eval_results): calculates aggregated results

### Create the evaluation pipeline

⚠️ If you want to use an **OpenAI** model for evaluation you will need an [API key](https://platform.openai.com/docs/quickstart) (see TODO below)

Change the arguments below if you want to run the pipeline with different parameters.

In [8]:
evaluation_args = {
    "retrieval_top_k": 2,
    "llm_module_name": "langchain.chat_models",
    "llm_class_name": "ChatOpenAI",
    "llm_kwargs": {
      "openai_api_key":"" ,   # TODO: Update with your key or use a different model
      "model_name" : "gpt-3.5-turbo"
    },
    "evaluation_metrics": ["context_precision", "context_relevancy"]
}

We begin by initializing our pipeline.

In [16]:
import pyarrow as pa
from fondant.pipeline import Pipeline
evaluation_pipeline = Pipeline(
        name="evaluation-pipeline",
        description="Pipeline to evaluate a RAG solution",
        base_path=shared_args["base_path"],
)


We have created a set of evaluation questions which we will use to evaluate the retrieval performance of the RAG system. Therefore, we need to load the CSV file containing the questions. We are going to use a reusable component for this task, `load_from_csv`.

In [17]:
evaluation_set_filename = "wikitext_1000_q.csv"

load_from_csv = evaluation_pipeline.read(
    "load_from_csv",
    arguments={
        "dataset_uri": "/evaldata/" + evaluation_set_filename,
        # mounted dir from within docker as extra_volumes
        "column_separator": ";",
    },
    produces={
        "question": pa.string(),
    },
)

Afterward, we are going to embed our questions and retrieve answers from the database. Here we will once again use the reusable `embed_text` component.

In [21]:
from fondant.pipeline import Resources
embed_text_op = load_from_csv.apply(
    "embed_text",
    arguments={
        "model_provider": shared_args["embed_model_provider"],
        "model": shared_args["embed_model"]
    },
    consumes={
        "text": "question",
    },
    resources=Resources(
        accelerator_number=number_of_accelerators,
        accelerator_name=accelerator_name,
    ),
    cluster_type="local" if number_of_accelerators is not None else "default",
)

TODO: describe retrieve chunk component and ragas eval component

In [19]:
from components.retrieve_from_weaviate import RetrieveFromWeaviateComponent
from components.evaluate_ragas import RagasEvaluator
from components.aggregrate_eval_results import AggregateResults

retrieve_chunks = embed_text_op.apply(
    RetrieveFromWeaviateComponent,
    arguments={
        "weaviate_url": shared_args["weaviate_url"],
        "class_name": shared_args["weaviate_url"],
        "top_k": 2
    },
    cache=False,
)

retriever_eval = retrieve_chunks.apply(
    RagasEvaluator,
    arguments={
        "llm_module_name": evaluation_args["llm_module_name"],
        "llm_class_name": evaluation_args["llm_class_name"],
        "llm_kwargs": evaluation_args["llm_kwargs"],
    }
)

retriever_eval.apply(
    AggregateResults
)

 Consumes: {'question': {'type': 'string'}, 'embedding': {'type': 'array', 'items': {'type': 'float32'}}}
 Consumes: {'question': {'type': 'string'}, 'embedding': {'type': 'array', 'items': {'type': 'float32'}}, 'retrieved_chunks': {'type': 'array', 'items': {'type': 'string'}}, 'context_precision': {'type': 'float32'}, 'context_relevancy': {'type': 'float32'}}


#### Run the evaluation pipeline

In [20]:
import os
if utils.check_weaviate_class_exists(
    local_weaviate_client,
    shared_args["weaviate_class"]
): 
    runner = DockerRunner()
    extra_volumes = [str(os.path.join(os.path.abspath('.'), "evaluation_datasets")) + ":/evaldata"]
    runner.run(evaluation_pipeline, extra_volumes=extra_volumes)

INFO:root:Class Pipeline1 exists in Weaviate.


INFO:root:Found reference to un-compiled pipeline... compiling
INFO:fondant.pipeline.compiler:Compiling evaluation-pipeline to .fondant/compose.yaml
INFO:fondant.pipeline.compiler:Base path found on local system, setting up ./data as mount volume
INFO:fondant.pipeline.pipeline:Sorting pipeline component graph topologically.
INFO:fondant.pipeline.pipeline:All pipeline component specifications match.
INFO:fondant.pipeline.compiler:Compiling service for load_from_csv
INFO:fondant.pipeline.compiler:Compiling service for embed_text
INFO:fondant.pipeline.compiler:Compiling service for retrievefromweaviatecomponent
INFO:fondant.pipeline.compiler:Compiling service for ragasevaluator
INFO:fondant.pipeline.compiler:Compiling service for aggregateresults
INFO:fondant.pipeline.compiler:Successfully compiled to .fondant/compose.yaml
 retrievefromweaviatecomponent Pulling 
 embed_text Pulling 
 load_from_csv Pulling 
 aggregateresults Pulling 
 ragasevaluator Pulling 


Starting pipeline run...


 ragasevaluator Pulled 
 embed_text Pulled 
 load_from_csv Pulled 
 aggregateresults Pulled 
 retrievefromweaviatecomponent Pulled 
 Container evaluation-pipeline-load_from_csv-1  Recreate
 Container evaluation-pipeline-load_from_csv-1  Recreated
 Container evaluation-pipeline-embed_text-1  Recreate
 Container evaluation-pipeline-embed_text-1  Recreated
 Container evaluation-pipeline-retrievefromweaviatecomponent-1  Recreate
 Container evaluation-pipeline-retrievefromweaviatecomponent-1  Recreated
 Container evaluation-pipeline-ragasevaluator-1  Recreate
 Container evaluation-pipeline-ragasevaluator-1  Recreated
 Container evaluation-pipeline-aggregateresults-1  Recreate
 Container evaluation-pipeline-aggregateresults-1  Recreated


Attaching to evaluation-pipeline-aggregateresults-1, evaluation-pipeline-embed_text-1, evaluation-pipeline-load_from_csv-1, evaluation-pipeline-ragasevaluator-1, evaluation-pipeline-retrievefromweaviatecomponent-1


...

evaluation-pipeline-ragasevaluator-1                 | [2024-02-06 12:29:49,340 | fondant.cli | INFO] Component `RagasEvaluator` found in module main
evaluation-pipeline-ragasevaluator-1                 | [2024-02-06 12:29:49,344 | fondant.component.executor | INFO] Dask default local mode will be used for further executions.Our current supported options are limited to 'local' and 'default'.
evaluation-pipeline-ragasevaluator-1                 | [2024-02-06 12:29:49,349 | fondant.component.executor | INFO] Previous component `retrievefromweaviatecomponent` is not cached. Invalidating cache for current and subsequent components
evaluation-pipeline-ragasevaluator-1                 | [2024-02-06 12:29:49,349 | fondant.component.executor | INFO] Caching disabled for the component
evaluation-pipeline-ragasevaluator-1                 | [2024-02-06 12:29:49,349 | root | INFO] Executing component
evaluation-pipeline-ragasevaluator-1                 | /usr/local/lib/python3.11/site-packages/lan

[                                        ] | 0% Completed | 629.17 us
[                                        ] | 0% Completed | 101.66 ms
[                                        ] | 0% Completed | 202.07 ms


evaluation-pipeline-ragasevaluator-1                 | Traceback (most recent call last):
evaluation-pipeline-ragasevaluator-1                 |   File "/usr/local/bin/fondant", line 8, in <module>
evaluation-pipeline-ragasevaluator-1                 |     sys.exit(entrypoint())
evaluation-pipeline-ragasevaluator-1                 |              ^^^^^^^^^^^^
evaluation-pipeline-ragasevaluator-1                 |   File "/usr/local/lib/python3.11/site-packages/fondant/cli.py", line 89, in entrypoint
evaluation-pipeline-ragasevaluator-1                 |     args.func(args)
evaluation-pipeline-ragasevaluator-1                 |   File "/usr/local/lib/python3.11/site-packages/fondant/cli.py", line 711, in execute
evaluation-pipeline-ragasevaluator-1                 |     executor.execute(component)
evaluation-pipeline-ragasevaluator-1                 |   File "/usr/local/lib/python3.11/site-packages/fondant/component/executor.py", line 383, in execute
evaluation-pipeline-ragasevaluator-1 

Finished pipeline run.


service "ragasevaluator" didn't complete successfully: exit 1


#### Show evaluation results

In [ ]:
utils.get_metrics_latest_run(base_path=BASE_PATH)

## Explore data

You can also check your data and results at each step in the pipelines using the **Fondant data explorer**. The first time you run the data explorer, you need to download the docker image which may take a minute. Then you can access the data explorer at: **http://localhost:8501/**

Enjoy the exploration! 🍫 

Press the ◼️ in the notebook toolbar to **stop the explorer**.

In [ ]:
from fondant.explore import run_explorer_app

run_explorer_app(base_path=BASE_PATH)

To stop the Explore, run the cell below.

In [ ]:
from fondant.explore import stop_explorer_app

stop_explorer_app()

## Clean up your environment

After your pipeline run successfully, you can **clean up** your environment and stop the weaviate database.

In [ ]:
!docker compose -f weaviate/docker-compose.yaml down

## Feedback

Please share your experience or **let us know how we can improve** through our 
* [**Discord**](https://discord.gg/HnTdWhydGp) 
* [**GitHub**](https://github.com/ml6team/fondant)

And of course feel free to give us a [**star** ⭐](https://github.com/ml6team/fondant) if you like what we are doing!